In [ ]:
!nvidia-smi

Fri Dec  8 10:32:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 400, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 400 (delta 95), reused 52 (delta 51), pack-reused 269
Receiving objects: 100% (400/400), 109.76 KiB | 884.00 KiB/s, done.
Resolving deltas: 100% (197/197), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.8 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.6/502.6 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import cuml

In [ ]:
from cuml.ensemble import RandomForestClassifier


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
from cuml import LogisticRegression
from cuml import SVC
from cuml import KNeighborsClassifier

In [ ]:
from xgboost import XGBClassifier

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score

In [ ]:
# loading dataset
dataset = pd.read_csv("/content/drive/MyDrive/LJP_2023/Embeddings/Doc2Vec/Exp2/Generic train - civil test/train_dataset.csv")
print(len(dataset))
dataset.rename(columns = {'judgement':'text'}, inplace = True)
dataset = dataset[dataset['label'] != "Other" ]
dataset["label"] = dataset["label"].replace("Accepted", 1)
dataset["label"] = dataset["label"].replace("Rejected", 0)

#filter dataset
dataset.label.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset = dataset.dropna(axis=0, subset=['text'])
dataset = dataset.dropna(axis=0, subset=['label'])
dataset['label'] = dataset['label'].astype(int)
dataset.reset_index(drop=True, inplace=True)
y_train = dataset['label']
print(y_train) #should have only the label column

41342
0        1
1        1
2        1
3        1
4        0
        ..
41337    0
41338    0
41339    1
41340    0
41341    1
Name: label, Length: 41342, dtype: int64


In [ ]:
# loading dataset
dataset = pd.read_csv("/content/drive/MyDrive/LJP_2023/Embeddings/Doc2Vec/Exp2/Generic train - civil test/testcivil.csv")
print(len(dataset))
dataset.rename(columns = {'judgement':'text'}, inplace = True)
dataset = dataset[dataset['label'] != "Other" ]
dataset["label"] = dataset["label"].replace("Accepted", 1)
dataset["label"] = dataset["label"].replace("Rejected", 0)

#filter dataset
dataset.label.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset = dataset.dropna(axis=0, subset=['text'])
dataset = dataset.dropna(axis=0, subset=['label'])
dataset['label'] = dataset['label'].astype(int)
dataset.reset_index(drop=True, inplace=True)
y_test = dataset['label']
print(y_test) #should have only the label column

1000
0      0
1      0
2      0
3      1
4      1
      ..
995    0
996    1
997    0
998    0
999    1
Name: label, Length: 1000, dtype: int64


In [ ]:
x_train=np.load("/content/drive/MyDrive/LJP_2023/Embeddings/Doc2Vec/Exp2/Generic train - civil test/train_doc2vec.npy")

In [ ]:
x_test=np.load("/content/drive/MyDrive/LJP_2023/Embeddings/Doc2Vec/Exp2/Generic train - civil test/test_doc2vec_civil.npy")

# Generic train - civil test

## Random Forest

In [ ]:
param_grid = {
    'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450]
}

# Create GridSearchCV and fit the data
grid_search = GridSearchCV(RandomForestClassifier(), param_grid=param_grid)
grid_search.fit(x_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [100, 150, 200, 250, 300, 350, 400,
                                          450]})

In [ ]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

best_rf = grid_search.best_estimator_
best_rf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_rf.predict(x_test)

# Calculate accuracy, precision, and recall scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Best Parameters: {'n_estimators': 300}
Best Score: 0.588650107383728
Accuracy: 0.72
Precision: 0.8163934426229508
Recall: 0.5264270613107822


## ExtraTreesClassifier

In [ ]:
param_grid = {
    'n_estimators': [400]
}

# Create GridSearchCV and fit the data
grid_search = GridSearchCV(ExtraTreesClassifier(n_jobs=-1), param_grid=param_grid)
grid_search.fit(x_train, y_train)

GridSearchCV(estimator=ExtraTreesClassifier(n_jobs=-1),
             param_grid={'n_estimators': [400]})

In [ ]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

best_rf = grid_search.best_estimator_
best_rf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_rf.predict(x_test)

# Calculate accuracy, precision, and recall scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Best Parameters: {'n_estimators': 400}
Best Score: 0.6145313986530441
Accuracy: 0.841
Precision: 0.8792270531400966
Recall: 0.7695560253699789


## XGBoost

In [ ]:
param_grid = {
    'max_depth': [3, 5, 7, 9]
}

# Create GridSearchCV and fit the data
grid_search = GridSearchCV(XGBClassifier(device='cuda'), param_grid=param_grid)
grid_search.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [21:17:44] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'max_depth': [3, 5, 7, 9]})

In [ ]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

best_rf = grid_search.best_estimator_
best_rf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_rf.predict(x_test)

# Calculate accuracy, precision, and recall scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Best Parameters: {'max_depth': 9}
Best Score: 0.5715728471697001
Accuracy: 0.697
Precision: 0.7043269230769231
Recall: 0.6194503171247357


## KNeighborsClassifier

In [ ]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9]
}

# Create GridSearchCV and fit the data
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid)
grid_search.fit(x_train, y_train)

[I] [21:20:06.197348] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [21:20:06.203810] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [21:20:07.148540] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [21:20:07.468224] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [21:20:07.784562] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [21:20:08.102760] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [21:20:08.419288] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [21:20:08.742552] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [21:20:09.063897] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [21:20:09.411798] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [21:20:09.735535] Unused keyword parameter: n_jobs during cuML estimator ini

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9]})

In [ ]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

best_rf = grid_search.best_estimator_
best_rf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_rf.predict(x_test)

# Calculate accuracy, precision, and recall scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Best Parameters: {'n_neighbors': 3}
Best Score: 0.576410436630249
Accuracy: 0.725
Precision: 0.7171052631578947
Recall: 0.6913319238900634


## QDA

In [ ]:
param_grid = {
    # 'reg_param': [0.001, 0.01, 0.1, 1.0]
}

# Create GridSearchCV and fit the data
grid_search = GridSearchCV(QuadraticDiscriminantAnalysis(), param_grid=param_grid)
grid_search.fit(x_train, y_train)

GridSearchCV(estimator=QuadraticDiscriminantAnalysis(), param_grid={})

In [ ]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

best_rf = grid_search.best_estimator_
best_rf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_rf.predict(x_test)

# Calculate accuracy, precision, and recall scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Best Parameters: {}
Best Score: 0.6362036693959515
Accuracy: 0.905
Precision: 0.8764940239043825
Recall: 0.9302325581395349


# Generic train - criminal test

In [ ]:
# loading dataset
dataset = pd.read_csv("/content/drive/MyDrive/LJP_2023/Embeddings/Doc2Vec/Exp2/Generic train - criminal test/train_dataset.csv")
print(len(dataset))
dataset.rename(columns = {'judgement':'text'}, inplace = True)
dataset = dataset[dataset['label'] != "Other" ]
dataset["label"] = dataset["label"].replace("Accepted", 1)
dataset["label"] = dataset["label"].replace("Rejected", 0)

#filter dataset
dataset.label.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset = dataset.dropna(axis=0, subset=['text'])
dataset = dataset.dropna(axis=0, subset=['label'])
dataset['label'] = dataset['label'].astype(int)
dataset.reset_index(drop=True, inplace=True)
y_train = dataset['label']
print(y_train) #should have only the label column

41342
0        1
1        1
2        1
3        1
4        0
        ..
41337    0
41338    0
41339    0
41340    1
41341    0
Name: label, Length: 41342, dtype: int64


In [ ]:
# loading dataset
dataset = pd.read_csv("/content/drive/MyDrive/LJP_2023/Embeddings/Doc2Vec/Exp2/Generic train - criminal test/testcriminal.csv")
print(len(dataset))
dataset.rename(columns = {'judgement':'text'}, inplace = True)
dataset = dataset[dataset['label'] != "Other" ]
dataset["label"] = dataset["label"].replace("Accepted", 1)
dataset["label"] = dataset["label"].replace("Rejected", 0)

#filter dataset
dataset.label.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset = dataset.dropna(axis=0, subset=['text'])
dataset = dataset.dropna(axis=0, subset=['label'])
dataset['label'] = dataset['label'].astype(int)
dataset.reset_index(drop=True, inplace=True)
y_test = dataset['label']
print(y_test) #should have only the label column

1000
0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    1
999    1
Name: label, Length: 1000, dtype: int64


In [ ]:
x_train=np.load("/content/drive/MyDrive/LJP_2023/Embeddings/Doc2Vec/Exp2/Generic train - criminal test/train_doc2vec.npy")

In [ ]:
x_test=np.load("/content/drive/MyDrive/LJP_2023/Embeddings/Doc2Vec/Exp2/Generic train - criminal test/test_doc2vec_criminal.npy")

## Random Forest

In [ ]:
param_grid = {
    'n_estimators': [100, 150, 200, 250, 300, 350, 400, 450]
}

# Create GridSearchCV and fit the data
grid_search = GridSearchCV(RandomForestClassifier(), param_grid=param_grid)
grid_search.fit(x_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'n_estimators': [100, 150, 200, 250, 300, 350, 400,
                                          450]})

In [ ]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

best_rf = grid_search.best_estimator_
best_rf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_rf.predict(x_test)

# Calculate accuracy, precision, and recall scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Best Parameters: {'n_estimators': 450}
Best Score: 0.5901014089584351
Accuracy: 0.701
Precision: 0.7777777777777778
Recall: 0.4880174291938998


## ExtraTreesClassifier

In [ ]:
param_grid = {
    'n_estimators': [400]
}

# Create GridSearchCV and fit the data
grid_search = GridSearchCV(ExtraTreesClassifier(n_jobs=-1), param_grid=param_grid)
grid_search.fit(x_train, y_train)

GridSearchCV(estimator=ExtraTreesClassifier(n_jobs=-1),
             param_grid={'n_estimators': [400]})

In [ ]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

best_rf = grid_search.best_estimator_
best_rf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_rf.predict(x_test)

# Calculate accuracy, precision, and recall scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Best Parameters: {'n_estimators': 400}
Best Score: 0.6165630452287596
Accuracy: 0.764
Precision: 0.7911227154046997
Recall: 0.6601307189542484


## XGBoost

In [ ]:
param_grid = {
    'max_depth': [3, 5, 7, 9]
}

# Create GridSearchCV and fit the data
grid_search = GridSearchCV(XGBClassifier(device='cuda'), param_grid=param_grid)
grid_search.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [14:02:43] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'max_depth': [3, 5, 7, 9]})

In [ ]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

best_rf = grid_search.best_estimator_
best_rf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_rf.predict(x_test)

# Calculate accuracy, precision, and recall scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Best Parameters: {'max_depth': 9}
Best Score: 0.5695896969012971
Accuracy: 0.645
Precision: 0.6293532338308457
Recall: 0.55119825708061


## KNeighborsClassifier

In [ ]:
param_grid = {
    'n_neighbors': [3, 5, 7, 9]
}

# Create GridSearchCV and fit the data
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid)
grid_search.fit(x_train, y_train)

[I] [14:04:59.201887] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:04:59.216492] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:05:00.280837] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:05:00.620733] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:05:00.957597] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:05:01.305863] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:05:01.643643] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:05:01.977557] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:05:02.336829] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:05:02.685301] Unused keyword parameter: n_jobs during cuML estimator initialization
[I] [14:05:03.038599] Unused keyword parameter: n_jobs during cuML estimator ini

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9]})

In [ ]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

best_rf = grid_search.best_estimator_
best_rf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_rf.predict(x_test)

# Calculate accuracy, precision, and recall scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Best Parameters: {'n_neighbors': 3}
Best Score: 0.5762169361114502
Accuracy: 0.694
Precision: 0.6652267818574514
Recall: 0.6710239651416122


## QDA

In [ ]:
param_grid = {
    # 'reg_param': [0.001, 0.01, 0.1, 1.0]
}

# Create GridSearchCV and fit the data
grid_search = GridSearchCV(QuadraticDiscriminantAnalysis(), param_grid=param_grid)
grid_search.fit(x_train, y_train)

GridSearchCV(estimator=QuadraticDiscriminantAnalysis(), param_grid={})

In [ ]:
# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

best_rf = grid_search.best_estimator_
best_rf.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_rf.predict(x_test)

# Calculate accuracy, precision, and recall scores
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Best Parameters: {}
Best Score: 0.6362522271354304
Accuracy: 0.842
Precision: 0.8003992015968064
Recall: 0.8736383442265795
